In [4]:
import os, sys, numpy, pyRofex
from configparser import ConfigParser
from matplotlib.pyplot import figure, subplots, style
from pandas import Series, DataFrame, Timestamp, Timedelta
from pandas import to_datetime, date_range, merge, concat
from pandas import Index, MultiIndex, TimedeltaIndex
style.use("dark_background")

creds = ConfigParser()
creds.read("../auth/credentials.ini")
creds = dict(creds["ROFEX"])
creds["environment"] = pyRofex.Environment.REMARKET

In [6]:
client = pyRofex.initialize(**creds)
client

ApiException: Authentication fails. Incorrect User or Password